In [2]:
import csv
import json
import pprint


Файлы именуются в виде "Фамилия\_Группа\_Вар\_номер.ipynb". Пример: "Милованов\_ПИ1-2\_Вар\_1.ipynb"

Не забываем про try-except. Везде, где вы считаете, что может вылететь ошибка, вставляем try except

Даны файлы students.csv, results.csv, groups.csv, subjects.csv, teachers.csv. Разделены ";"

Файл groups.csv имеет столбцы id, text_name, entry_year, где id - уникиальный идентификатор группы, text_name - наименование группы, entry_year - год поступления. 

students.csv имеет следующие столбцы: id, first_name, last_name, group_id, где id - уникальный идентификатор студента (номер зачетной книжки), first_name - Имя, last_name - фамилия, group_id - идентификатор группы, в которой он учится.

result.csv имеет следующие столбцы: id - уникальный идентификатор записи, subject_id - идентификатор учебного предмета, student_id - уникальный идентификатор студента, att1 - результат первой аттестации, att2 - результат второй аттестации, exam - результат экзаменнационной или зачетной работы, total - общий результат, teacher_id - уникальный идентификатор преподавателя.

subjects.csv имеет следующие столбцы: id - уникальный идентификатор предмета, subject_name - наименование предмета.

teachers.csv: id - уникальный идентификатор преподавателя, first_name - Имя, last_name - фамилия, middle_name - отчество

1. Написать функцию, которая принимает id преподавателя и id группы. Функция возвращает False, если данный преподаватель не преподавал у данной группы, None, если такого прподавателя не существует, иначе возвращается словарь, в котором ключами являются наименования предметов, а значениями словари, которые хранят в себе информации о количестве студентов, сдавших на 5, 4, 3 и 2 по данному предмету.

In [35]:
def funk_1(teacher_id, group_id):
    teachers = []
    with open('teachers.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] == "i":
                continue
            teachers.append({"id":int(row[0]), 
                            "first_name":row[1], 
                            "last_name":row[2],
                            "middle_name":row[3]
                            })
    subjects = {}
    with open('subjects.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] == "i":
                continue
            subjects[int(row[0])] = row[1]


    check = 0
    for i in teachers:
        if i["id"] == teacher_id:
            check = 1
    if check == 0:
        return("None")
    
    students = []
    with open('students.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] == "i":
                continue
            if int(row[3]) == group_id:
                students.append(int(row[0]))
    
    dic = {}
    with open('results.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] == "i":
                continue
            if int(row[2]) in students and int(row[3]) == teacher_id:
                if subjects[int(row[1])] not in dic:
                    dic[subjects[int(row[1])]] = {"5":0, "4":0, "3":0, "2":0}
                if int(row[7]) >= 85:
                    dic[subjects[int(row[1])]]["5"] += 1
                elif int(row[7]) >= 75:
                    dic[subjects[int(row[1])]]["4"] += 1
                elif int(row[7]) >= 51:
                    dic[subjects[int(row[1])]]["3"] += 1
                else:
                    dic[subjects[int(row[1])]]["2"] += 1
        if dic == {}:
            return ("False")
    return(dic)

funk_1(5, 4)

{'Реинжиниринг бизнес-процессов': {'5': 2, '4': 9, '3': 9, '2': 6}}

2. Написать функцию, которая принимает ФИО преподавателя и необязательный параметр to_json, который по умолчанию равен False и принимает имя файла. Если указан параметр to_json, то он должен сохранить итоговый результат в данный файл и вернуть значение True, иначе просто вернуть словарь. В словаре ключами являются наименования групп, в которых он преподавал, а значениям - результат выполнения первой функции.

In [32]:
def funk_2(FIO, to_json = False):
    teacher_id = -1
    with open('teachers.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] == "i":
                continue
            if (FIO.split()[0] == row[1] and
               FIO.split()[1] == row[2] and
               FIO.split()[2] == row[3]):
                teacher_id = int(row[0])
            elif teacher_id == -1:
                return 'Препод отсутствует'

            
    students_dic = {}
    with open('students.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] == "i":
                continue
            students_dic[row[0]] = int(row[3])
    
    students_id = []
    for k in students_dic.keys():
        students_id.append(k) 
            
    groups_id = []
    with open('results.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] == "i":
                continue
            if int(row[3]) == teacher_id and row[2] in students_id:
                if students_dic[row[2]] not in groups_id:
                    groups_id.append(students_dic[row[2]])
                    
    groups_dic = {}
    with open('groups.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] == "i":
                continue
            groups_dic[int(row[0])] = row[1]
            
    dic2 = {}
    for e in groups_id:
        dic2[groups_dic[e]] = funk_1(teacher_id, e)
        
    if to_json:
        with open('{name}'.format(name = to_json), 'w') as f:
            json.dump(dic2, f)
        return True
    else:
        return dic2
    

pprint.pprint(funk_2('Милованов Даниил Михайлович'))
        

    
                

{'ПИ1-1': {'Web-программирование': {'2': 3, '3': 12, '4': 8, '5': 4},
           'Микроэкономика': {'2': 6, '3': 5, '4': 13, '5': 3},
           'Философия': {'2': 5, '3': 13, '4': 8, '5': 1}},
 'ПИ1-2': {'Математический анализ': {'2': 2, '3': 6, '4': 13, '5': 4},
           'Философия': {'2': 8, '3': 6, '4': 11, '5': 0}},
 'ПИ1-3': {'Дискретная математика': {'2': 4, '3': 11, '4': 6, '5': 5}},
 'ПИ1-4': {'Линейная алгебра': {'2': 2, '3': 12, '4': 9, '5': 3},
           'Современные языки программирования': {'2': 3,
                                                  '3': 9,
                                                  '4': 11,
                                                  '5': 3}},
 'ПИ4-1': {'Макроэкономика': {'2': 6, '3': 5, '4': 5, '5': 2},
           'Математический анализ': {'2': 3, '3': 4, '4': 9, '5': 2}},
 'ПИ4-2': {'Вычислительные системы и сетевые телекоммуникации': {'2': 4,
                                                                 '3': 9,
                      

3. Реализовать функцию, которая принимает параметры entry_year - год поступления, subject_name - наименование предмета и необязаиельный параметр to_json, по умолчанию равный False. Функция должна возвращать словарь со статистикой по группам по данному предмету. Ключами словаря являются id группы, а значениями словари. Внутренний словарь имеет ключи group_name (наименование группы), stats (статистика). Значением ключа статистика является словарь, который возвращает количество студентов получившие оценку 5, количество студентов получившие оценку 4, количество студентов получившие оценку 3, количество студентов получившие оценку 2. Если параметр to_json указан, то сохраняет полученный словарь в указанный файл и возвращает True, иначе просто возвращает словарь.

In [10]:
def funk_3(entry_year, subject_name, to_json = False):    
    subjects = {}
    with open('subjects.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] != "i":
                subjects[row[1]] = int(row[0])
        
            
    groups_dic = {}
    with open('groups.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] != "i":
                groups_dic[int(row[0])] = [row[1], int(row[2])]
    
    dicti3 = {}       
    for k, v in groups_dic.items():
        if v[1] == entry_year: 
            dicti3[k] = {"group_name":v[0], "stats":{"5":0,"4":0,"3":0,"2":0}}
    
    
            students = []
            with open('students.csv', 'r') as f:
                csv_reader = csv.reader(f, delimiter=';')
                for row in csv_reader:
                    if row[0][0] != "i":
                        if int(row[3]) == k:
                            students.append(int(row[0]))
                    
            with open('results.csv', 'r') as f:
                csv_reader2 = csv.reader(f, delimiter=';')
                for row2 in csv_reader2:
                    if row2[0][0] != "i":
                    
                        if int(row2[2]) in students and subjects[subject_name] == int(row2[1]):
                            if int(row2[7]) >= 85:
                                dicti3[k]["stats"]["5"] += 1
                            elif int(row2[7]) >= 75:
                                dicti3[k]["stats"]["4"] += 1
                            elif int(row2[7]) >= 51:
                                dicti3[k]["stats"]["3"] += 1
                            else:
                                dicti3[k]["stats"]["2"] += 1
                                
    if to_json:
        with open('{name}'.format(name = to_json), 'w') as f:
            json.dump(dicti3, f)
        return True
    else:
        return dicti3
                    
pprint.pprint(funk_3(2018, 'Линейная алгебра'))                
            
    

{1: {'group_name': 'ПИ1-1', 'stats': {'2': 11, '3': 14, '4': 2, '5': 0}},
 2: {'group_name': 'ПИ1-2', 'stats': {'2': 13, '3': 9, '4': 2, '5': 1}},
 3: {'group_name': 'ПИ1-3', 'stats': {'2': 11, '3': 9, '4': 4, '5': 2}},
 4: {'group_name': 'ПИ1-4', 'stats': {'2': 9, '3': 12, '4': 3, '5': 2}},
 5: {'group_name': 'ПМ1-1', 'stats': {'2': 15, '3': 5, '4': 5, '5': 1}},
 6: {'group_name': 'ПМ1-2', 'stats': {'2': 5, '3': 14, '4': 4, '5': 1}},
 7: {'group_name': 'ПМ1-3', 'stats': {'2': 5, '3': 15, '4': 3, '5': 1}},
 8: {'group_name': 'ПМ1-4', 'stats': {'2': 8, '3': 14, '4': 3, '5': 0}},
 10: {'group_name': 'ПМ1-5', 'stats': {'2': 12, '3': 11, '4': 0, '5': 2}}}


1. Реализовать функцию, которая по уникальному идентификатору студента возвращает его сумму баллов по каждому из контрольных мероприятий, в том числе total.

In [9]:
def Gospodi_pomogi(x):
    lst1 = []
    stud_id = str(x)
    (att1, att2, exam, total) = (0, 0, 0, 0)
    with open('results.csv', 'r') as csvfile:
        csv_reader = csv.DictReader(csvfile, delimiter=';')
        for row in csv_reader:               
                if row['student_id'] == stud_id:
                        att1 += int(row['att1'])
                        att2 += int(row['att2'])           
                        exam += int(row['exam'])
                        total += int(row['total'])
        dicti = {stud_id: (att1, att2, exam, total)}
    return(dicti)

                                   

In [10]:
Gospodi_pomogi(151301)

{'151301': (252, 279, 510, 1041)}

2. Реализовать функцию, которая принимает параметры thread - поток в виде (ПИ2018), где буквами является наименование направления, а число - год поступления, необязательный параметр test, который по умолчанию равен total (список возможных значений att1, att2, exam, total), а также необязательный параметр to_json=False. Если параметр указан, то результат выполнения функции должен записываться в файл и функция возвращает True, если такого потока не существует, то возвращает None, иначе возвращает результат функции. Результатом выполнения функции является словарь с рейтингом студентов по указанному из периодов в параметре test. Ключом словаря является место студента в рейтинге, значением - словарь с информацией из файла students.csv, а также информацию, которая возвращается после выполнения первой функции.

3. Написать функцию, которая принимает  имя группы, год поступления и наименование предмета. Также функция принимает необязательный параметр to_json, который по умолчанию равен False и принимает имя файла. Функция должна вернуть словарь, которая содержит оценки всех студентов данной группы по данному предмету. Ключом является id студента, значением словарь, который содержит полное имя студента и итоговый балл.  Если to_json указывает имя файла, то сохранить результаты в файл и функция возвращает значение True, иначе возвращает получившийся словарь.

In [14]:
def funk_6(group_name, entry_year, subject_name, to_json = False): 
    with open('groups.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] != "i":
                if group_name == row[1] and entry_year == int(row[2]):
                    group_id = int(row[0])
                    
    subjects = {}
    with open('subjects.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] != "i":
                subjects[row[1]] = int(row[0])               
   
    students = {}
    with open('students.csv', 'r') as f:
        csv_reader = csv.reader(f, delimiter=';')
        for row in csv_reader:
            if row[0][0] != "i":
                if int(row[3]) == group_id:
                    students[int(row[0])] = row[1] + " " + row[2]


    dicti6 = {}       
    for k, v in students.items(): 
        dicti6[k] = {"name":v, "total":0}
        with open('results.csv', 'r') as f:
            csv_reader = csv.reader(f, delimiter=';')
            for row in csv_reader:
                if row[0][0] != "i":
                    if int(row[2]) == k and subjects[subject_name] == int(row[1]):
                        dicti6[k]["total"] = int(row[7])
        
    if to_json:
        with open('{name}'.format(name = to_json), 'w') as f:
            json.dump(dicti6, f)
        return True
    else:
        return dicti6
    
pprint.pprint(funk_6("ПИ1-4", 2018, "Базы данных"))
    

{180401: {'name': 'Арутюнян Давид\xa0', 'total': 67},
 180402: {'name': 'Балабанов Владислав\xa0', 'total': 45},
 180403: {'name': 'Беленькая Софья\xa0', 'total': 33},
 180404: {'name': 'Борисенко Никита\xa0', 'total': 38},
 180405: {'name': 'Габов Антон\xa0', 'total': 60},
 180406: {'name': 'Геворкян Альберт\xa0', 'total': 62},
 180407: {'name': 'Горячев Рустам\xa0', 'total': 69},
 180408: {'name': 'Гушан Наталия\xa0', 'total': 81},
 180409: {'name': 'Егоров Денис\xa0', 'total': 73},
 180410: {'name': 'Ефремов Сергей', 'total': 74},
 180411: {'name': 'Илюшко Сергей\xa0', 'total': 32},
 180412: {'name': 'Калинин Максим\xa0', 'total': 57},
 180413: {'name': 'Коваленко Глеб\xa0', 'total': 39},
 180414: {'name': 'Комиссаров Александр\xa0', 'total': 65},
 180415: {'name': 'Корчагин Александр', 'total': 39},
 180416: {'name': 'Кручинина Яна', 'total': 24},
 180417: {'name': 'Мамуков Сармат\xa0', 'total': 70},
 180418: {'name': 'Молчанова Вероника\xa0', 'total': 58},
 180419: {'name': 'Образ